## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-24-10-50-48 +0000,nypost,Trump says FEMA is ‘fully prepared’ for Winter...,https://nypost.com/2026/01/24/us-news/trump-sa...
1,2026-01-24-10-43-14 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/24/weathe...
2,2026-01-24-10-02-13 +0000,nyt,How a Photographer Stumbled Upon a Key Picture...,https://www.nytimes.com/2026/01/24/insider/how...
3,2026-01-24-10-02-00 +0000,nyt,"Despite Trump’s Words, China and Russia Are No...",https://www.nytimes.com/2026/01/24/us/politics...
4,2026-01-24-10-01-57 +0000,nyt,Will Leaving My Terrible Job Make Me Look Flaky?,https://www.nytimes.com/2026/01/24/business/wi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2488/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,56
249,ice,18
23,greenland,16
184,new,11
21,russia,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
258,2026-01-23-15-04-40 +0000,wapo,"On Greenland, Europe stood up, Trump blinked, ...",https://www.washingtonpost.com/world/2026/01/2...,94
66,2026-01-24-02-00-00 +0000,wsj,Trump’s aggressive deployment of ICE has thrus...,https://www.wsj.com/politics/policy/democrat-r...,94
226,2026-01-23-17-00-09 +0000,latimes,Trump lawyers urge Supreme Court to block Cali...,https://www.latimes.com/politics/story/2026-01...,93
3,2026-01-24-10-02-00 +0000,nyt,"Despite Trump’s Words, China and Russia Are No...",https://www.nytimes.com/2026/01/24/us/politics...,92
239,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
258,94,2026-01-23-15-04-40 +0000,wapo,"On Greenland, Europe stood up, Trump blinked, ...",https://www.washingtonpost.com/world/2026/01/2...
212,53,2026-01-23-18-10-10 +0000,nypost,DHS officials debunk viral claims ICE used 5-y...,https://nypost.com/2026/01/23/us-news/dhs-offi...
166,43,2026-01-23-20-39-22 +0000,nypost,"US, Ukraine and Russia hold first-ever trilate...",https://nypost.com/2026/01/23/world-news/us-uk...
103,39,2026-01-23-23-34-51 +0000,nypost,Ex-NJ Gov. Phil Murphy granted clemency to mom...,https://nypost.com/2026/01/23/us-news/murphy-g...
235,38,2026-01-23-16-18-59 +0000,nypost,NYC judge paves way for delivery customers to ...,https://nypost.com/2026/01/23/business/judge-d...
239,35,2026-01-23-16-09-21 +0000,nyt,Trump Withdraws Carney’s Invitation for Canada...,https://www.nytimes.com/2026/01/23/us/politics...
74,34,2026-01-24-01-11-10 +0000,nyt,"Ryan Wedding, Canadian Ex-Snowboarder Accused ...",https://www.nytimes.com/2026/01/23/world/canad...
36,33,2026-01-24-05-31-24 +0000,startribune,"The latest: Thousands march, scores of Minneso...",https://www.startribune.com/ice-raids-minnesot...
194,32,2026-01-23-18-59-46 +0000,startribune,Suspect in little girl’s abduction was at her ...,https://www.startribune.com/abducted-zimmerman...
43,31,2026-01-24-04-00-00 +0000,wsj,China Built a Vast Oil Stake in Venezuela. Now...,https://www.wsj.com/world/americas/china-venez...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
